# **Deploying a trained model on an embedded device to work with a camera.**

In [ ]:
# Import necessary libraries
import cv2  # OpenCV library for video and image processing
import torch  # PyTorch, the underlying framework for the YOLO model
import argparse  # For parsing command-line arguments
from ultralytics import YOLO  # YOLO model library
import RPi.GPIO as GPIO  # For controlling Raspberry Pi GPIO pins
import time  # For timing control

# Motor control setup
GPIO.setmode(GPIO.BCM)
MOTOR_PINS = {
    "recyclable": 17,
    "hazardous": 18,
    "biodegradable": 22,
    "general": 23
}

# Initialize GPIO pins
for pin in MOTOR_PINS.values():
    GPIO.setup(pin, GPIO.OUT)
    GPIO.output(pin, GPIO.LOW)

def open_bin(bin_type, duration=3):
    """Control servo to open specific trash bin"""
    print(f"Opening {bin_type} bin")
    GPIO.output(MOTOR_PINS[bin_type], GPIO.HIGH)
    time.sleep(duration)
    GPIO.output(MOTOR_PINS[bin_type], GPIO.LOW)

def parse_args():
    """
    Parses command-line arguments for the script.
    """
    parser = argparse.ArgumentParser(description="Smart Trash Bin Detection System")
    parser.add_argument('--model', type=str, default='best.pt', help='Path to YOLO model')
    parser.add_argument('--conf', type=float, default=0.8, help='Confidence threshold (default: 0.8)')
    parser.add_argument('--camera', type=int, default=0, help='Camera index (default: 0)')
    return parser.parse_args()

def main():
    """
    Main function to run the object detection program.
    """
    args = parse_args()

    # Class mapping dictionary
    CLASS_MAP = {
        0: "recyclable",    # glass
        1: "recyclable",    # metal
        2: "recyclable",    # paper
        3: "recyclable",    # plastic
        4: "hazardous",     # battery
        5: "recyclable",    # cardboard
        6: "general",       # clothes
        7: "biodegradable", # organic
        8: "general"        # shoes
    }

    # Load the YOLO model
    print(f"Loading model {args.model}...")
    model = YOLO(args.model)

    # Open the webcam
    cap = cv2.VideoCapture(args.camera)
    if not cap.isOpened():
        print(f"Error: Could not open webcam at index {args.camera}")
        return

    # Main processing loop
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Error: Could not read frame")
            break

        frame_resized = cv2.resize(frame, (640, 480))
        results = model(frame_resized)

        detected_classes = set()  # To track detected classes in current frame

        for result in results:
            for box in result.boxes:
                conf = box.conf[0].item()
                cls = int(box.cls[0].item())

                # Only process detections above confidence threshold
                if conf >= args.conf:
                    detected_classes.add(cls)

                    # Get coordinates and draw bounding box
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    label = f"{CLASS_MAP.get(cls, 'unknown')} ({conf:.2f})"

                    cv2.rectangle(frame_resized, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    cv2.putText(frame_resized, label, (x1, y1 - 10),
                               cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

        # Open corresponding bin if any detection meets confidence threshold
        if detected_classes:
            # Get the most confident detection
            main_class = max(detected_classes, key=lambda x: box.conf[0].item())
            bin_type = CLASS_MAP.get(main_class, "general")
            open_bin(bin_type)

        cv2.imshow("Smart Trash Bin Detection", frame_resized)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Cleanup
    cap.release()
    cv2.destroyAllWindows()
    GPIO.cleanup()

if __name__ == "__main__":
    main()

โครงสร้างของโค้ด
การนำเข้าไลบรารี (Imports):

cv2: คือไลบรารี OpenCV ใช้สำหรับจัดการกับวิดีโอและรูปภาพ เช่น การจับภาพจากกล้อง การปรับขนาดภาพ และการวาดกรอบ

torch: คือไลบรารี PyTorch ซึ่งเป็นพื้นฐานของโมเดล YOLO ที่ใช้ในการคำนวณและประมวลผลข้อมูล

argparse: ใช้สำหรับจัดการกับอาร์กิวเมนต์ที่ส่งเข้ามาทางคอมมานด์ไลน์ ทำให้สามารถกำหนดค่าต่างๆ เช่น ชื่อโมเดลหรือความแม่นยำได้ง่ายขึ้น

ultralytics: เป็นไลบรารีที่ใช้โมเดล YOLO ทำให้การเรียกใช้งานโมเดลทำได้ง่าย

ฟังก์ชัน parse_args():

ฟังก์ชันนี้จะกำหนดพารามิเตอร์ที่สามารถปรับเปลี่ยนได้เมื่อรันโค้ด เช่น:

--model: ชื่อไฟล์โมเดล YOLO (ค่าเริ่มต้นคือ yolov8n.pt)

--conf: ค่าความแม่นยำขั้นต่ำที่ยอมรับได้ (confidence threshold) เพื่อกรองผลลัพธ์ที่ไม่น่าเชื่อถือ (ค่าเริ่มต้นคือ 0.25)

--camera: หมายเลขกล้องที่ต้องการใช้งาน (ค่าเริ่มต้นคือ 0 ซึ่งมักจะเป็นกล้องหลัก)

ฟังก์ชัน main():

เป็นฟังก์ชันหลักที่รันโปรแกรม

args = parse_args(): เรียกใช้ฟังก์ชันด้านบนเพื่อรับค่าพารามิเตอร์ต่างๆ

model = YOLO(args.model): โหลดโมเดล YOLO ที่ระบุ

cap = cv2.VideoCapture(args.camera): เปิดการใช้งานกล้องเว็บแคม

while True:: สร้างลูปไม่รู้จบเพื่อประมวลผลวิดีโอทีละเฟรม

ret, frame = cap.read(): อ่านเฟรมจากกล้อง

frame_resized = cv2.resize(...): ปรับขนาดเฟรมให้เล็กลงเพื่อความเร็วในการประมวลผล

results = model(frame_resized): ส่งเฟรมที่ปรับขนาดแล้วเข้าไปในโมเดล YOLO เพื่อตรวจจับวัตถุ

for result in results:: วนลูปเพื่อดูผลลัพธ์ของแต่ละวัตถุที่ตรวจจับได้

cv2.rectangle(...) และ cv2.putText(...): วาดกรอบสี่เหลี่ยมและข้อความระบุคลาสและความแม่นยำลงบนเฟรม

cv2.imshow(...): แสดงผลเฟรมที่มีกรอบวัตถุ

if cv2.waitKey(1) & 0xFF == ord('q'):: กดปุ่ม q เพื่อออกจากโปรแกรม

cap.release() และ cv2.destroyAllWindows(): ปิดการใช้งานกล้องและหน้าต่างแสดงผลเมื่อโปรแกรมจบการทำงาน